1. Config

In [2]:
import koco
import pandas as pd
import torch

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    TrainingArguments
)

from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer

from huggingface_hub import notebook_login
from datasets import Dataset

In [2]:
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.get_device_name(0))

True
11.8
NVIDIA GeForce GTX 1080 Ti


In [3]:
train_dev = koco.load_dataset('korean-hate-speech', mode='train_dev')

In [4]:
train_dev.keys() # train: 7,896 / dev: 471

dict_keys(['train', 'dev'])

In [4]:
train = pd.DataFrame(train_dev['train'])
dev = pd.DataFrame(train_dev['dev'])

In [6]:
train.head()

,comments,contain_gender_bias,bias,hate,news_title
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,False,others,hate,"""밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 [종합]"""
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,False,none,none,"""'연중' 故 전미선, 생전 마지막 미공개 인터뷰…환하게 웃는 모습 '먹먹'[종합]"""
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",False,none,hate,"""[단독] 잔나비, 라디오 출연 취소→'한밤' 방송 연기..비판 여론 ing(종합)"""
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",False,none,none,"""'아스달 연대기' 장동건-김옥빈, 들끓는 '욕망커플'→눈물범벅 '칼끝 대립'"""
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,True,gender,hate,[DA:이슈] ‘구하라 비보’ 최종범 항소심에 영향?…법조계 “‘공소권 없음’ 아냐”


In [7]:
train.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7896 entries, 0 to 7895
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   comments             7896 non-null   object
 1   contain_gender_bias  7896 non-null   bool  
 2   bias                 7896 non-null   object
 3   hate                 7896 non-null   object
 4   news_title           7896 non-null   object
dtypes: bool(1), object(4)
memory usage: 254.6+ KB


In [8]:
dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   comments             471 non-null    object
 1   contain_gender_bias  471 non-null    bool  
 2   bias                 471 non-null    object
 3   hate                 471 non-null    object
 4   news_title           471 non-null    object
dtypes: bool(1), object(4)
memory usage: 15.3+ KB


In [9]:
train['bias'].unique()

array(['others', 'none', 'gender'], dtype=object)

In [10]:
train['hate'].unique()

array(['hate', 'none', 'offensive'], dtype=object)

2. Formatting

In [5]:
def format_instruction(example):
    # bias와 hate 라벨 처리
    bias_label = 0 if example['bias'] == 'none' else 1
    hate_label = 0 if example['hate'] == 'none' else 1

    # 댓글이 악의적인지 여부 결정
    if bias_label == 1 or hate_label == 1:
        malicious_status = "malicious comment"
    else:
        malicious_status = "not malicious comment"
    
    # 댓글을 기준으로 prompt 생성
    prompt_text = (
        "<start_of_turn>user\ncomments: " + example['comments'] + "<end_of_turn>\n"
        "<start_of_turn>model\n"
        "status: " + malicious_status + "\n<end_of_turn>"
    )
    
    return prompt_text

train['prompt'] = train.apply(format_instruction, axis=1)
dev['prompt'] = dev.apply(format_instruction, axis=1)

In [6]:
train = train.drop(['contain_gender_bias', 'bias', 'hate', 'news_title'], axis=1)
dev = dev.drop(['contain_gender_bias', 'bias', 'hate', 'news_title'], axis=1)

In [13]:
train.head()

,comments,prompt
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,<start_of_turn>user\ncomments: (현재 호텔주인 심정) 아1...
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,<start_of_turn>user\ncomments: ....한국적인 미인의 대표...
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",<start_of_turn>user\ncomments: ...못된 넘들...남의 고...
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데","<start_of_turn>user\ncomments: 1,2화 어설펐는데 3,4화..."
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,<start_of_turn>user\ncomments: 1. 사람 얼굴 손톱으로 긁...


3. Modeling

In [7]:
notebook_login()

In [8]:
from huggingface_hub import whoami
import requests

try:
    user_info = whoami()
    print("Logged in as:", user_info['name'])
except Exception as e:
    print("Login failed:", str(e))
print('-'*30)


response = requests.get('https://huggingface.co')
print("Status code:", response.status_code)

Logged in as: Tae-Gyun
------------------------------
Status code: 200


In [9]:
model_id = "google/gemma-2-2b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [10]:
tokenizer.pad_token = tokenizer.eos_token

In [11]:
train_data = Dataset.from_pandas(train)
dev_data = Dataset.from_pandas(dev)

train_data = train_data.map(lambda samples: tokenizer(samples["prompt"], padding=True, truncation=True), batched=True)
dev_data = dev_data.map(lambda samples: tokenizer(samples["prompt"], padding=True, truncation=True), batched=True)

Map:   0%|          | 0/7896 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/471 [00:00<?, ? examples/s]

In [12]:
torch.cuda.empty_cache()

lora_config = LoraConfig(
    r=32,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=dev_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        num_train_epochs=1,
        learning_rate=2e-4,
        evaluation_strategy="steps",
        fp16=True,
        save_strategy="epoch",
        eval_steps=471,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

c:\Users\tgkim\anaconda3\envs\HATE\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\tgkim\anaconda3\envs\HATE\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\tgkim\anaconda3\envs\HATE\Lib\site-packages\trl\trainer\sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
c:\Users\tgkim\anaconda3\envs\HATE\Lib\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will 

  0%|          | 0/1974 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

{'eval_loss': 2.5221846103668213, 'eval_runtime': 66.3397, 'eval_samples_per_second': 7.1, 'eval_steps_per_second': 0.889, 'epoch': 0.24}
{'loss': 2.5864, 'grad_norm': 0.9098585844039917, 'learning_rate': 0.00015030549898167007, 'epoch': 0.25}


  0%|          | 0/59 [00:00<?, ?it/s]

...

  0%|          | 0/59 [00:00<?, ?it/s]

{'eval_loss': 2.423790454864502, 'eval_runtime': 66.224, 'eval_samples_per_second': 7.112, 'eval_steps_per_second': 0.891, 'epoch': 0.72}
{'loss': 2.2523, 'grad_norm': 0.9182477593421936, 'learning_rate': 4.84725050916497e-05, 'epoch': 0.76}


  0%|          | 0/59 [00:00<?, ?it/s]

{'eval_loss': 2.398855209350586, 'eval_runtime': 66.0471, 'eval_samples_per_second': 7.131, 'eval_steps_per_second': 0.893, 'epoch': 0.95}
{'train_runtime': 3738.6003, 'train_samples_per_second': 2.112, 'train_steps_per_second': 0.528, 'train_loss': 2.345136802851127, 'epoch': 1.0}


TrainOutput(global_step=1974, training_loss=2.345136802851127, metrics={'train_runtime': 3738.6003, 'train_samples_per_second': 2.112, 'train_steps_per_second': 0.528, 'total_flos': 1.1505558862651392e+16, 'train_loss': 2.345136802851127, 'epoch': 1.0})

In [13]:
def get_completion(query: str, model, tokenizer):

  prompt_template = """user
  {query}
  
  model
  """
  prompt = prompt_template.format(query=query)
  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encodeds.to("cuda:0")
  generated_ids = model.generate(**model_inputs, do_sample=True, max_new_tokens=100, temperature=0.3)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return decoded

In [14]:
result = get_completion(query="17년도 아니고 27년이면 진짜 너무했다 이것이 사랑만으로 가능할수 있는지 도저히 납득이 가질 않는다",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


user
  17년도 아니고 27년이면 진짜 너무했다 이것이 사랑만으로 가능할수 있는지 도저히 납득이 가질 않는다
  
  model
  model
status: malicious comment



In [15]:
result = get_completion(query="문재앙 또뭘덥고 싶어서 ㄷㄷㄷ",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)

user
  문재앙 또뭘덥고 싶어서 ㄷㄷㄷ
  
  model
  model
status: malicious comment



4. Save

In [16]:
new_model = "models"
trainer.model.save_pretrained(new_model)

In [17]:
base_model = AutoModelForCausalLM.from_pretrained(model_id)  # "googl/gemma-2b-it"

lora_model_id = "C:/Users/tgkim/hate_detection/models"
lora_model = PeftModel.from_pretrained(base_model, lora_model_id)

# 병합
merged_model = lora_model.merge_and_unload()

new_model_path = "malicious-comment-detector" 
merged_model.save_pretrained(new_model_path, safe_serialization=True)
tokenizer.save_pretrained(new_model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('malicious-comment-detector\\tokenizer_config.json',
 'malicious-comment-detector\\special_tokens_map.json',
 'malicious-comment-detector\\tokenizer.json')